In [23]:
import pandas as pd

def remove_column(csv_file, column_name):
    df = pd.read_csv(csv_file)
    if column_name in df.columns:
        df = df.drop(columns=[column_name])
    df.to_csv(csv_file,index=False)

def add_data(database_file, csv_file, skip_new=False):
    new_data = pd.read_csv(csv_file)
    df = pd.read_csv(database_file)
    for column in new_data.columns:
        if column not in df.columns:
            df[column] = None
    for index, row in new_data.iterrows():
        matching_index = df[df['Wikidata ID'] == row[0]].index
        if not skip_new:
            if matching_index.empty:
                df = df.append(row, ignore_index=True)
            else:
                df.at[matching_index[0], 'Context'] = row[1]
        elif not matching_index.empty:
            df.at[matching_index[0], 'Context'] = row[1]
    df.to_csv(database_file, index=False)


In [21]:
import pandas as pd

def add_data(database_file, csv_file, skip_new=False):
    new_data = pd.read_csv(csv_file)
    df = pd.read_csv(database_file)
    
    # Ensure all columns in new_data exist in df
    for column in new_data.columns:
        if column not in df.columns:
            df[column] = None
    
    # DataFrame to hold new rows to be added
    new_rows = pd.DataFrame(columns=df.columns)
    
    for index, row in new_data.iterrows():
        matching_index = df[df['Wikidata ID'] == row[0]].index
        if not skip_new:
            if matching_index.empty:
                new_rows = pd.concat([new_rows, pd.DataFrame([row], columns=new_data.columns)], ignore_index=True)
            else:
                df.at[matching_index[0], 'Context'] = row[1]
        elif not matching_index.empty:
            df.at[matching_index[0], 'Context'] = row[1]
    
    # Concatenate the new rows to the existing DataFrame
    df = pd.concat([df, new_rows], ignore_index=True)
    
    df.to_csv(database_file, index=False)

In [25]:
remove_column("/Users/lucyhorowitz/Documents/GitHub/MathGloss/database copy.csv", "nLab")
add_data("/Users/lucyhorowitz/Documents/GitHub/MathGloss/database copy.csv", "/Users/lucyhorowitz/Documents/GitHub/MathGloss/nlab_mappings.csv",skip_new=True)

/var/folders/zt/k_rggns533n6tv1w6sj042yc0000gn/T/ipykernel_70114/703644504.py:16: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  matching_index = df[df['Wikidata ID'] == row[0]].index
/var/folders/zt/k_rggns533n6tv1w6sj042yc0000gn/T/ipykernel_70114/703644504.py:23: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df.at[matching_index[0], 'Context'] = row[1]


In [7]:
import pandas as pd

# Load the existing cleaned database
df_cleaned = pd.read_csv('database_cleaned.csv')

# Load the new data from another CSV file
new_data = pd.read_csv('/Users/lucyhorowitz/Documents/MathGloss/ZZZnewstuff/context_mappings.csv')

# Ensure the new column "Context" exists in df_cleaned
df_cleaned['Context'] = None

# Update the "Context" column in df_cleaned based on the new data
for index, row in new_data.iterrows():
    matching_index = df_cleaned[df_cleaned['Wikidata ID'] == row[0]].index
    if not matching_index.empty:
        df_cleaned.at[matching_index[0], 'Context'] = row[1]

# Save the updated DataFrame to a CSV file
df_cleaned.to_csv('database_cleaned.csv', index=False)

/var/folders/zt/k_rggns533n6tv1w6sj042yc0000gn/T/ipykernel_33559/4075708671.py:14: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  matching_index = df_cleaned[df_cleaned['Wikidata ID'] == row[0]].index
/var/folders/zt/k_rggns533n6tv1w6sj042yc0000gn/T/ipykernel_33559/4075708671.py:16: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df_cleaned.at[matching_index[0], 'Context'] = row[1]


In [24]:
import os
import glob
import re

# Define the directory containing the markdown files
directory = '/Users/lucyhorowitz/Documents/GitHub/MathGloss/context'

# Get a list of all markdown files in the directory
markdown_files = glob.glob(os.path.join(directory, '*.md'))

# Iterate over each markdown file
for filepath in markdown_files:
    # Read the content of the file
    with open(filepath, 'r', encoding='utf-8') as file:
        content = file.read()
    
    # Replace {{,} with ,
    updated_content = re.sub(r'{{,}', ',', content)

    # Write the updated content back to the file
    with open(filepath, 'w', encoding='utf-8') as file:
        file.write(updated_content)

In [31]:
import re
import pandas as pd
from neo4j import GraphDatabase
import requests

# Read the CSV file into a DataFrame
df = pd.read_csv('database.csv')

# Connect to the Neo4j database
uri = "bolt://localhost:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "12345678"))

def find_node(tx, wikidata_id):
    query = f"""
        MATCH (n:Concept {{wikidata_id: $wikidata_id}})
        RETURN n.wikidata_name AS property
        """
    result = tx.run(query, wikidata_id=wikidata_id)
    record = result.single()
    if record:
        return record['property']
    return None

def get_wikidata_label(wikidata_id):
    wikidata_id_2 = re.search(r'Q\d+', wikidata_id).group()
    with driver.session() as session:
        wikiname = session.execute_read(find_node, wikidata_id_2)
        if wikiname:
            print(f"Found label in Neo4j for Wikidata ID {wikidata_id_2}: {wikiname}")
            return wikiname
        else:
            url = f"https://www.wikidata.org/wiki/Special:EntityData/{wikidata_id_2}.json"
            response = requests.get(url)
            if response.status_code == 200:
                data = response.json()
                if wikidata_id_2 in data['entities']:
                    labels = data['entities'][wikidata_id_2]['labels']
                    if 'en' in labels:
                        label = labels['en']['value']
                        print(f"Found label in Wikidata for Wikidata ID {wikidata_id_2}: {label}")
                        return label
                    else:
                        print(f"No English label found for Wikidata ID {wikidata_id_2}")
                        return None
                else:
                    print(f"Wikidata ID {wikidata_id_2} not found in the response")
                    return None
            else:
                print(f"Failed to retrieve label for Wikidata ID {wikidata_id_2}")
                return None

# Apply the function to create the 'Wikidata Label' column
df['Wikidata Label'] = df['Wikidata ID'].apply(get_wikidata_label)

# Ensure 'Wikidata Label' is to the right of 'Wikidata ID'
wikidata_id_index = df.columns.get_loc('Wikidata ID')
df.insert(wikidata_id_index + 1, 'Wikidata Label', df.pop('Wikidata Label'))

print(df)

# Close the driver after all operations are completed
driver.close()

Found label in Wikidata for Wikidata ID Q14481419: Zero–one law
Found label in Neo4j for Wikidata ID Q181296: abelian group
Found label in Neo4j for Wikidata ID Q318598: abelianization
Found label in Neo4j for Wikidata ID Q20827138: Absolutely continuous measures
Found label in Neo4j for Wikidata ID Q332504: absolute continuity
Found label in Wikidata for Wikidata ID Q332465: absolute convergence
Found label in Neo4j for Wikidata ID Q120812: absolute value
Found label in Wikidata for Wikidata ID Q4681343: additive category
Found label in Neo4j for Wikidata ID Q2858846: transpose of a linear map
Found label in Neo4j for Wikidata ID Q4379157: adjoint representation of a Lie algebra
Found label in Neo4j for Wikidata ID Q1017106: adjoint representation
Found label in Neo4j for Wikidata ID Q225107: adjugate matrix
Found label in Wikidata for Wikidata ID Q382504: affine hull
Found label in Wikidata for Wikidata ID Q382698: affine space
Found label in Neo4j for Wikidata ID Q382497: affine tra

In [ ]:
# Define a function to get the label from the Wikidata ID
def get_wikidata_label(wikidata_id):
    match = re.search(r'Q\d+', wikidata_id)
    if match:
        return match.group()
    return None

# Apply the function to create a new column 'Wikidata Label'
df['Wikidata Label'] = df['Wikidata ID'].apply(get_wikidata_label)

# Save the updated DataFrame to a CSV file


In [32]:
df.to_csv('database.csv', index=False)

In [3]:
input_file = '/Users/lucyhorowitz/Documents/GitHub/MathGloss/alignments/france-maps.txt'
output_file = '/Users/lucyhorowitz/Documents/GitHub/MathGloss/alignments/france-maps-formatted.csv'

# Read the input file
with open(input_file, 'r', encoding='utf-8') as file:
    lines = file.readlines()

# Process each line and format it
formatted_lines = []
for line in lines:
    line2 = line.split(': ')
    line2 = [thing.strip() for thing in line2]
    if line:
        wikidata_link = f"[{line2[1]}](https://www.wikidata.org/wiki/{line2[1]})"
        formatted_link = f"[{line2[0]}"
        formatted_lines.append(f"{wikidata_link},{formatted_link}")

# Write the formatted lines to the output file
with open(output_file, 'w', encoding='utf-8') as file:
    file.write('\n'.join(formatted_lines))

print(f"Formatted data has been written to {output_file}")

Formatted data has been written to /Users/lucyhorowitz/Documents/GitHub/MathGloss/alignments/france-maps-formatted.csv
